In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from mmm_eval.data.synth_data_generator import generate_data
from mmm_eval.configs import PyMCConfig
from pymc_marketing.prior import Prior
from mmm_eval import run_evaluation

In [2]:
data = generate_data()

X = data.drop(columns=["revenue","quantity"])
y = data["quantity"]

In [3]:
base_model = MMM(
    date_column="date_week" ,
    channel_columns=["channel_1","channel_2"],
    adstock=GeometricAdstock(l_max=4),
    saturation=LogisticSaturation()
)

# base_model.fit(X=X, y=y, chains=4, target_accept=0.85)



In [4]:
# We dont need X and y - we'll get those from the input data!
fit_kwargs = { 
    "chains": 4,
    "target_accept": 0.85,
}

base_config = PyMCConfig.from_model_object(base_model, fit_kwargs=fit_kwargs, response_column="quantity", revenue_column="revenue")

In [5]:
base_config

PyMCConfig(revenue_column='revenue', response_column='quantity', pymc_model_config=PyMCModelSchema(date_column='date_week', channel_columns=['channel_1', 'channel_2'], adstock=GeometricAdstock(prefix='adstock', l_max=4, normalize=True, mode='After', priors={'alpha': Prior("Beta", alpha=1, beta=3, dims="channel")}), saturation=LogisticSaturation(prefix='saturation', priors={'lam': Prior("Gamma", alpha=3, beta=1, dims="channel"), 'beta': Prior("HalfNormal", sigma=2, dims="channel")}), time_varying_intercept=False, time_varying_media=False, sampler_config={}, validate_data=True, control_columns=None, yearly_seasonality=None, adstock_first=True, dag=None, treatment_nodes=None, outcome_node=None), fit_config=PyMCFitSchema(draws=None, tune=None, chains=4, target_accept=0.85, random_seed=None, progressbar=False, return_inferencedata=True), date_column='date_week', channel_columns=['channel_1', 'channel_2'], control_columns=None)

In [6]:
results = run_evaluation("pymc-marketing", data, base_config) 

2025-06-26 17:02:21,749 - mmm_eval.core.validation_test_orchestrator - INFO - Running test: accuracy
2025-06-26 17:02:21,750 - mmm_eval.core.base_validation_test - INFO - Splitting data into train and test sets for accuracy test
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
2025-06-26 17:02:29,358 - pymc.stats.convergence - ERROR - There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
2025-06-26 17:02:29,363 - mmm_eval.adapters.experimental.pymc - INFO - Computing channel contributions...
2025-06-26 17:02:29,372 - mmm_eval.adapters.experimental.pymc - INFO - Predicting...
2025-06-26 17:02:29,800 - mmm_eval.core.validation_tests - INFO - Saving the test results for accuracy test
2025-06-26 17:02:29,801 - mmm_eval.core.validation_test_orchestrator - INFO - Running test: cross_validation
2025-06-26 17:02:29,801 - mmm_eval.core.base_validation_test - INFO - Splitting data into train and test sets for cross_validation test
20